In [16]:
import pandas as pd

# # 최대 행 수를 None으로 설정하여 모든 행이 출력되도록 함
# pd.set_option('display.max_rows', None)

# # 최대 열 수를 None으로 설정하여 모든 열이 출력되도록 함
# pd.set_option('display.max_columns', None)

# # 행과 열의 출력 옵션을 원래 상태로 되돌림
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')

In [15]:
# CSV 파일 불러오기
df = pd.read_csv(".dataset_total.csv")

# 데이터프레임 확인
df.head() # 상위 5개 행 출력


FileNotFoundError: [Errno 2] No such file or directory: '.tmp/dataset/dataset_total.csv'

In [ ]:
df.shape

(244280, 114)

In [ ]:
# 데이터셋의 기본 정보 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244280 entries, 0 to 244279
Columns: 114 entries, SK_ID_CURR to Credit_Utilization_Ratio
dtypes: float64(63), int64(41), object(10)
memory usage: 212.5+ MB


In [ ]:
# 각 열의 결측치 수 확인
missing_values = df.isnull().sum()
missing_values[missing_values > 0]

Series([], dtype: int64)

In [ ]:
# 데이터 타입 확인
df.dtypes

SK_ID_CURR                           int64
TARGET                               int64
NAME_CONTRACT_TYPE                  object
CODE_GENDER                         object
FLAG_OWN_CAR                        object
                                    ...   
Dependents_Index                   float64
Income_to_Dependents_Ratio         float64
Debt_to_Income_Ratio               float64
Debt_Repayment_Capability_Index    float64
Credit_Utilization_Ratio           float64
Length: 114, dtype: object

In [ ]:
# 타겟 변수의 분포 확인
target_counts = df['TARGET'].value_counts()
print(target_counts)

TARGET
0    225248
1     19032
Name: count, dtype: int64


현상이 형 요청 사항 시작

In [ ]:
names = [
    "강민호", "김현빈", "김희교", "박동현", "박선애", "성원석", "신동훈", "신지민", 
    "오손빈", "위혜인", "이나은", "이동현", "이승준", "이승훈", "이유리", "이주영", 
    "이현상", "장성민", "전영욱", "정명훈", "정지영", "정혜란", "최우진", "최호윤", 
    "피상민"
]


In [ ]:
# LOAN_COUNT가 0 이상인 데이터와 0인 데이터를 각각 필터링
loan_count_positive = df[df['LOAN_COUNT'] > 0]
loan_count_zero = df[df['LOAN_COUNT'] == 0]

In [ ]:
# TARGET의 비율에 맞춰 LOAN_COUNT가 0 이상인 데이터와 0인 데이터에서 각각 샘플링

# TARGET이 0인 경우 12개, 1인 경우 13개를 각각 샘플링 (1:1 비율로 나누어지도록)

sample_positive_0 = loan_count_positive[loan_count_positive['TARGET'] == 0].sample(n=12, random_state=42)
sample_positive_1 = loan_count_positive[loan_count_positive['TARGET'] == 1].sample(n=13, random_state=42)

sample_zero_0 = loan_count_zero[loan_count_zero['TARGET'] == 0].sample(n=12, random_state=42)
sample_zero_1 = loan_count_zero[loan_count_zero['TARGET'] == 1].sample(n=13, random_state=42)


In [ ]:
# 1:1 비율로 데이터 결합 (12개씩 0과 1에 대해서 합침)
sampled_data_positive = pd.concat([sample_positive_0, sample_positive_1]).reset_index(drop=True)
sampled_data_zero = pd.concat([sample_zero_0, sample_zero_1]).reset_index(drop=True)


In [ ]:
# 각각에서 12개씩을 결합하여 총 24개의 데이터를 만들고 남은 1개는 TARGET의 0과 1 비율에 맞게 선택
sampled_data = pd.concat([sampled_data_positive.sample(n=12, random_state=42),  sampled_data_zero.sample(n=12, random_state=42)]).reset_index(drop=True)

# 이미 사용된 SK_ID_CURR를 제외한 남은 샘플을 선택하기 위해 필터링
used_sk_ids = sampled_data['SK_ID_CURR'].tolist()
remaining_samples = pd.concat([sample_positive_0, sample_positive_1, sample_zero_0, sample_zero_1])
remaining_samples = remaining_samples[~remaining_samples['SK_ID_CURR'].isin(used_sk_ids)]

# 남은 1개의 샘플을 TARGET 비율에 맞게 선택 (분포에 맞추기 위해)
target_ratio = 19032 / 244280

if target_ratio > 0.5:
    additional_sample = remaining_samples[remaining_samples['TARGET'] == 1].sample(n=1, random_state=42)
else:
    additional_sample = remaining_samples[remaining_samples['TARGET'] == 0].sample(n=1, random_state=42)



In [ ]:
# 최종 샘플 데이터에 추가하여 25개로 만듦
sampled_data = pd.concat([sampled_data, additional_sample]).reset_index(drop=True)


In [ ]:
# 샘플링한 데이터의 name 컬럼의 값을 names 리스트의 이름으로 교체
sampled_data['name'] = names

In [ ]:
sampled_data

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,AMT_APPLICATION_SUM,AMT_APPLICATION_MEAN,RATE_DOWN_PAYMENT_MEAN,IS_REVOLVING_LOAN,Dependents_Index,Income_to_Dependents_Ratio,Debt_to_Income_Ratio,Debt_Repayment_Capability_Index,Credit_Utilization_Ratio,name
0,342407,0,Cash loans,M,Y,Y,0,360000.0,1223010.0,48631.5,...,502866.0,1.676220e+05,0.050004,0.0,0.0,360000.0,2.218750,0.040709,0.347725,강민호
1,247197,1,Cash loans,F,Y,N,0,225000.0,942300.0,26041.5,...,3196701.0,6.393402e+05,0.099205,0.0,0.0,225000.0,1.679543,0.159080,0.283952,김현빈
2,388971,0,Revolving loans,M,N,Y,0,270000.0,270000.0,13500.0,...,234846.0,3.354943e+04,0.135609,1.0,0.0,270000.0,0.036515,0.025958,0.136114,김희교
3,342954,1,Cash loans,F,N,Y,0,202500.0,835380.0,35523.0,...,71681.4,2.389380e+04,0.106632,0.0,0.0,202500.0,1.316024,0.018697,0.596455,박동현
4,293026,0,Cash loans,F,Y,N,0,112500.0,450000.0,16294.5,...,13115965.5,1.008920e+06,0.173986,0.0,0.0,112500.0,0.000000,0.385823,0.000000,박선애
5,219676,0,Cash loans,F,Y,Y,0,126000.0,360000.0,23436.0,...,131346.0,6.567300e+04,0.000000,0.0,0.0,126000.0,0.000000,0.051686,0.000000,성원석
6,390619,1,Cash loans,M,Y,N,0,270000.0,790830.0,62482.5,...,1499908.5,9.999390e+04,0.004328,0.0,0.0,270000.0,2.211995,0.063638,0.558137,신동훈
7,307654,0,Revolving loans,F,Y,Y,1,225000.0,450000.0,22500.0,...,135009.0,6.750450e+04,0.000000,0.0,0.5,112500.0,0.000000,0.029178,0.000000,신지민
8,294801,1,Cash loans,F,Y,Y,0,90000.0,432567.0,29389.5,...,785025.0,2.616750e+05,0.000000,0.0,0.0,90000.0,0.249963,0.250583,0.064046,오손빈
9,381931,0,Revolving loans,F,N,N,2,121500.0,202500.0,10125.0,...,14602.5,1.460250e+04,0.000000,0.0,1.0,40500.0,0.135256,0.012316,0.275619,위혜인


In [ ]:
# name 컬럼을 SK_ID_CURR 뒤로 이동
columns = list(sampled_data.columns)
# 현재 컬럼들 중 name을 제외한 컬럼들
columns.remove('name')
# 새로운 컬럼 순서를 정의
new_columns = columns[:1] + ['name'] + columns[1:]

# 데이터프레임의 컬럼 순서를 변경
sampled_data = sampled_data[new_columns]

# 결과 확인
sampled_data

,SK_ID_CURR,name,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,...,AMT_APPLICATION_MAX,AMT_APPLICATION_SUM,AMT_APPLICATION_MEAN,RATE_DOWN_PAYMENT_MEAN,IS_REVOLVING_LOAN,Dependents_Index,Income_to_Dependents_Ratio,Debt_to_Income_Ratio,Debt_Repayment_Capability_Index,Credit_Utilization_Ratio
0,342407,강민호,0,Cash loans,M,Y,Y,0,360000.0,1223010.0,...,225000.0,502866.0,1.676220e+05,0.050004,0.0,0.0,360000.0,2.218750,0.040709,0.347725
1,247197,김현빈,1,Cash loans,F,Y,N,0,225000.0,942300.0,...,1012500.0,3196701.0,6.393402e+05,0.099205,0.0,0.0,225000.0,1.679543,0.159080,0.283952
2,388971,김희교,0,Revolving loans,M,N,Y,0,270000.0,270000.0,...,120195.0,234846.0,3.354943e+04,0.135609,1.0,0.0,270000.0,0.036515,0.025958,0.136114
3,342954,박동현,1,Cash loans,F,N,Y,0,202500.0,835380.0,...,26730.0,71681.4,2.389380e+04,0.106632,0.0,0.0,202500.0,1.316024,0.018697,0.596455
4,293026,박선애,0,Cash loans,F,Y,N,0,112500.0,450000.0,...,2250000.0,13115965.5,1.008920e+06,0.173986,0.0,0.0,112500.0,0.000000,0.385823,0.000000
5,219676,성원석,0,Cash loans,F,Y,Y,0,126000.0,360000.0,...,77346.0,131346.0,6.567300e+04,0.000000,0.0,0.0,126000.0,0.000000,0.051686,0.000000
6,390619,신동훈,1,Cash loans,M,Y,N,0,270000.0,790830.0,...,202351.5,1499908.5,9.999390e+04,0.004328,0.0,0.0,270000.0,2.211995,0.063638,0.558137
7,307654,신지민,0,Revolving loans,F,Y,Y,1,225000.0,450000.0,...,76950.0,135009.0,6.750450e+04,0.000000,0.0,0.5,112500.0,0.000000,0.029178,0.000000
8,294801,오손빈,1,Cash loans,F,Y,Y,0,90000.0,432567.0,...,765000.0,785025.0,2.616750e+05,0.000000,0.0,0.0,90000.0,0.249963,0.250583,0.064046
9,381931,위혜인,0,Revolving loans,F,N,N,2,121500.0,202500.0,...,14602.5,14602.5,1.460250e+04,0.000000,0.0,1.0,40500.0,0.135256,0.012316,0.275619


In [ ]:
# 데이터프레임을 CSV 파일로 저장
sampled_data.to_csv('sampled_data.csv', index=False, encoding='utf-8-sig')


# 칼럼별 고유값 추출 (원본)

In [ ]:
# 범주형 (object) 및 boolean 컬럼만 선택
categorical_columns = df.select_dtypes(include=['object', 'bool']).columns

# 각 범주형 컬럼의 고유값을 딕셔너리로 저장
unique_values_dict = {col: df[col].unique().tolist() for col in categorical_columns}

# 결과 출력
unique_values_dict

{'NAME_CONTRACT_TYPE': ['Cash loans', 'Revolving loans'],
 'CODE_GENDER': ['M', 'F', 'XNA'],
 'FLAG_OWN_CAR': ['N', 'Y'],
 'FLAG_OWN_REALTY': ['Y', 'N'],
 'NAME_TYPE_SUITE': ['Unaccompanied',
  'Spouse, partner',
  'Children',
  'Family',
  'Other_A',
  'Other_B',
  'Group of people'],
 'NAME_INCOME_TYPE': ['Working',
  'State servant',
  'Commercial associate',
  'Pensioner',
  'Unemployed',
  'Student',
  'Businessman'],
 'NAME_EDUCATION_TYPE': ['Secondary / secondary special',
  'Higher education',
  'Incomplete higher',
  'Lower secondary',
  'Academic degree'],
 'NAME_FAMILY_STATUS': ['Single / not married',
  'Married',
  'Widow',
  'Civil marriage',
  'Separated'],
 'NAME_HOUSING_TYPE': ['House / apartment',
  'Rented apartment',
  'Municipal apartment',
  'With parents',
  'Office apartment',
  'Co-op apartment'],
 'WEEKDAY_APPR_PROCESS_START': ['WEDNESDAY',
  'MONDAY',
  'SUNDAY',
  'SATURDAY',
  'FRIDAY',
  'THURSDAY',
  'TUESDAY']}

In [ ]:
import pandas as pd

# 주어진 딕셔너리
unique_values_dict = {
    'NAME_CONTRACT_TYPE': ['Cash loans', 'Revolving loans'],
    'CODE_GENDER': ['M', 'F', 'XNA'],
    'FLAG_OWN_CAR': ['N', 'Y'],
    'FLAG_OWN_REALTY': ['Y', 'N'],
    'NAME_TYPE_SUITE': ['Unaccompanied', 'Spouse, partner', 'Children', 'Family', 'Other_A', 'Other_B', 'Group of people'],
    'NAME_INCOME_TYPE': ['Working', 'State servant', 'Commercial associate', 'Pensioner', 'Unemployed', 'Student', 'Businessman'],
    'NAME_EDUCATION_TYPE': ['Secondary / secondary special', 'Higher education', 'Incomplete higher', 'Lower secondary', 'Academic degree'],
    'NAME_FAMILY_STATUS': ['Single / not married', 'Married', 'Widow', 'Civil marriage', 'Separated'],
    'NAME_HOUSING_TYPE': ['House / apartment', 'Rented apartment', 'Municipal apartment', 'With parents', 'Office apartment', 'Co-op apartment'],
    'WEEKDAY_APPR_PROCESS_START': ['WEDNESDAY', 'MONDAY', 'SUNDAY', 'SATURDAY', 'FRIDAY', 'THURSDAY', 'TUESDAY']
}

# 가장 긴 리스트의 길이를 찾음
max_length = max(len(values) for values in unique_values_dict.values())

# 각 리스트를 max_length에 맞추어 패딩
padded_dict = {key: values + [values[-1]] * (max_length - len(values)) for key, values in unique_values_dict.items()}

# 데이터프레임으로 변환
df_padded = pd.DataFrame(padded_dict)

# 결과 출력
df_padded

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,WEEKDAY_APPR_PROCESS_START
0,Cash loans,M,N,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,WEDNESDAY
1,Revolving loans,F,Y,N,"Spouse, partner",State servant,Higher education,Married,Rented apartment,MONDAY
2,Revolving loans,XNA,Y,N,Children,Commercial associate,Incomplete higher,Widow,Municipal apartment,SUNDAY
3,Revolving loans,XNA,Y,N,Family,Pensioner,Lower secondary,Civil marriage,With parents,SATURDAY
4,Revolving loans,XNA,Y,N,Other_A,Unemployed,Academic degree,Separated,Office apartment,FRIDAY
5,Revolving loans,XNA,Y,N,Other_B,Student,Academic degree,Separated,Co-op apartment,THURSDAY
6,Revolving loans,XNA,Y,N,Group of people,Businessman,Academic degree,Separated,Co-op apartment,TUESDAY


In [ ]:
len(unique_values_dict)

10

In [ ]:
max_length

7

In [ ]:
df_padded.shape

(7, 10)

In [ ]:
df_padded.to_csv('df_padded.csv', index=False, encoding='utf-8-sig')
